<a href="https://colab.research.google.com/github/jasmis1229/midterm_new/blob/main/%ED%8A%B8%EB%A0%8C%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ CBOW 실험 초기화

# 1. 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import re
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# 2. 데이터 불러오기
df = pd.read_csv('/content/신조어_스타일_패턴_완성본_UTF8SIG.csv')  # 파일명은 환경에 맞게 수정
